## Downloading images from Reddit

####References:
- https://github.com/mikf/gallery-dl
- https://www.reddit.com/r/DataHoarder/comments/k86s45/comment/gezgt44/
- https://www.reddit.com/r/Catsubs/wiki/index/#wiki_general_cats
- https://stackoverflow.com/a/47310093/19819055

#### Notes
1000 Images per sub should be fine

#### Baselines
Maybe use v1 and compare to v2


### Updates to this notebook
- added /top behind the url to sort posts by upvotes (might result in better image quality)
- ~~use Jupyter notebook's !{command} instead of subprocess.run() (should show more details of what's going on while waiting, since run() doesn't log command outputs to the notebook for some reason...~~
 turns out this was too spammy and slowed the whole browser down. reverting back...


# Setting up stuff

Installing required packages

In [ ]:
!python3 -m pip install -U gallery-dl

In [ ]:
conda install -c conda-forge imagemagick

Importing stuff

In [ ]:
from subprocess import run
from threading import Thread
from time import sleep
import os, shutil

In [ ]:
!magick

# Configuration Settings
- Amount
- Subreddits

In [ ]:
amount = 100#@param {type:"integer"}
subreddits_to_download="""
CreamsicleCats
IllegallySmolCats
GrumpyCats
norwegianForestCats
MaineCoons
Ragdolls
BlackCats
Torties
Torbie
CalicoKittys
tortico
MunchkinCats
Sphynx
PersianCat
RussianBlue
ScottishFold
SiameseCats
BengalCats
BritishShortHair
Savannah_Cats
Nebelung
SiberianCats
DevonRex
ExoticShorthair
TuxedoCats
OneOrangeBraincell
BurmeseCats
TwoFacedCats
turkishvan
orientalshorthair
flamepoints
standardissuecat
watercolorcats
piebaldcats
cowcats
blotchfacecats
SeniorCats
SleepingCats
FluffyCats
PirateKitties
"""

### Converting the string to a list

In [ ]:
subreddit_list = subreddits_to_download.strip().split("\n")
print(f"Number of Subreddits: {len(subreddit_list)}")
print(f"List Preview: {subreddit_list}")

## Finally downloading the data

In [ ]:
%cd ~/sagemaker 

from subprocess import DEVNULL

def download_images(subreddit_name):
  print(f"Executing run command for subreddit {subreddit_name}")

  url=f"https://www.reddit.com/r/{subreddit_name}/top/?t=all" 
  #/top/?t=all is required to make it sort by upvotes. This may naturally filter out some bad-quality images

  command = f'''gallery-dl --range 1-{amount} {url}'''#running the command
  run(command, shell=True, stdout=DEVNULL, stderr=DEVNULL)
  print(f"Done downloading {subreddit_name}")

for subreddit_name in subreddit_list:
  Thread(target=download_images, args=(subreddit_name,)).start()
  #sleep(1)
  #download_images(subreddit_name)


input("\nKeeping this cell active to monitor thread progress. Press enter when they are all done: ") #keep cell active to show output

## Renaming files (optional)

In [ ]:
%pwd

In [ ]:
%cd ~

In [ ]:
for folder in os.listdir("/home/studio-lab-user/sagemaker/gallery-dl/reddit/"):
  os.chdir(f"/home/studio-lab-user/sagemaker/gallery-dl/reddit/{folder}")
  filesInFolder = os.listdir()
  print(f"Detected {len(filesInFolder)} files in folder {folder}")
  #print(filesInFolder)
  for i in range(len(filesInFolder)):
    j = filesInFolder[i]
    if j.startswith("."):
      pass
    else:
      ext = j.split(".")[-1]
      os.rename(j, f"{str(i).zfill(4)}.{ext}")

## Compressing it a bit

In [ ]:
def resize_images(folder):
  os.chdir(f"/home/studio-lab-user/sagemaker/gallery-dl/reddit/{folder}")
  print(f"Starting resize for {folder}.")
  run("mogrify -format jpg -filter cubic -resize 1024x1024 *.*", shell=True, stdout=DEVNULL, stderr=DEVNULL)
  #resize the image using imagemagick

  print(f"Almost there, cleaning up {folder}")
  run("find . -type f ! -name '*.jpg' -delete", shell=True)

  print(f"Done resizing {folder}")

for folder in os.listdir("/home/studio-lab-user/sagemaker/gallery-dl/reddit/"):
  Thread(target=resize_images, args=(folder,)).start()
  #resize_images(folder)

print("Successfully launched the resizers, this will take a while... Maybe go grab a drink?")
input("\nKeeping this cell active to monitor thread progress. Press enter when they are all done: ")

### Zipping and obtaining the results

---



In [ ]:
%cd /home/studio-lab-user/sagemaker
!ls
%cd ./gallery-dl/reddit/
!rm cat-data.zip
!zip cat-data.zip -r . -0
#files.download("/content/cat-data.zip")

In [ ]:
%ls